---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [3]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs # No Directed graphs! =D

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [68]:
def graph_identification():
    
    # find the measures
    measures = {}
    index = 0
    for g in P1_Graphs:
        measures['g{}'.format(index)] = {
                                        'avg_clust': nx.average_clustering(g),
                                        'avg_sh_path': nx.average_shortest_path_length(g),
                                        'degree_hist': [degree for degree in nx.degree_histogram(g) if degree > 0]
                                        }   

        index += 1

    # find out the algorithms
    algorithms = []
    for g in measures:
        if len(measures[g]['degree_hist']) > 10:
            algorithms.append('PA')
        elif measures[g]['avg_clust'] <= 0.1:
            algorithms.append('SW_H')
        else:
            algorithms.append('SW_L')

    return algorithms

In [69]:
# graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be working with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [78]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [302]:
#

In [303]:
#

In [106]:
#

### Task:
- Predict a probability of the node receiving a management salary for nodes where ManagementSalary is missing
- The evaluation metric for this assignment is the Area Under the ROC Curve (AUC)

In [108]:
#

In [143]:
nx.is_connected(G)

False

In [304]:
#

In [169]:
df = pd.DataFrame(index=G.nodes())

In [170]:
df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
df['target'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))

In [148]:
# connections = []
# for node in G.nodes():
#     connections.append([e[1] for e in G.edges(node)])

# df['Connections'] = pd.Series(connections)

In [171]:
df['Degree'] = pd.Series(G.degree())
df['Clustering'] = pd.Series(nx.clustering(G))
df['DegreeCentrality'] = pd.Series(nx.degree_centrality(G))
df['BetweenessCentrality'] = pd.Series(nx.betweenness_centrality(G))
df['ClosenessCentrality'] = pd.Series(nx.closeness_centrality(G))
df['PageRank'] = pd.Series(nx.pagerank(G))

In [191]:
df_train = df[~pd.isnull(df['target'])]
df_train.head(3)

,Department,target,Degree,Clustering,DegreeCentrality,BetweenessCentrality,ClosenessCentrality,PageRank
0,1,0.0,44,0.276423,0.043825,0.001124,0.421991,0.001224
3,21,1.0,71,0.384910,0.070717,0.001654,0.441663,0.001833
4,21,1.0,96,0.318691,0.095618,0.005547,0.462152,0.002526


In [192]:
df_test = df[pd.isnull(df['target'])]
df_test.head(3)

,Department,target,Degree,Clustering,DegreeCentrality,BetweenessCentrality,ClosenessCentrality,PageRank
1,1,NaN,52,0.265306,0.051793,0.001195,0.422360,0.001426
2,21,NaN,95,0.297803,0.094622,0.006570,0.461490,0.002605
5,25,NaN,171,0.107002,0.170319,0.030995,0.501484,0.004914


In [256]:
len(df), len(df_train), len(df_test), (len(df_train) + len(df_test))

(1005, 753, 252, 1005)

In [305]:
#

In [258]:
from sklearn.model_selection import train_test_split, ShuffleSplit

In [287]:
# train_test_split

x_train, x_test, y_train, y_test = train_test_split(df_train.drop(labels=['target'], axis=1),
                                                    df_train['target'],
                                                    random_state=42)

x_train.shape, x_test.shape, y_train.shape, y_test.shape

((564, 7), (189, 7), (564,), (189,))

In [298]:
# shuffle split

rs = ShuffleSplit(n_splits=5, test_size=0.25, random_state=42)
indexes = {}
split = 0
for train_index, test_index in rs.split(df_train):
    indexes['split{}'.format(split)] = {'train': list(train_index), 'test': list(test_index)}
    split += 1

In [306]:
#

In [212]:
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process

In [307]:
#

In [122]:
#

In [121]:
#

In [ ]:
def salary_predictions():
    
    # Your Code Here
    
    return # Your Answer Here

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [ ]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [ ]:
def new_connections_predictions():
    
    # Your Code Here
    
    return # Your Answer Here